# Machine Learning Fairness Algorithms Evaluation

#### Fairness in Machine Learning
The aim of a fairness algorithm is to avoid the outcome decisions are being made unfairly to certain groups or individuals.
- Algorithms are made by humans and trained by data which may be biased
- However, there are many definitions of fairness that cannot be optimized at the same time


#### Learning fair representations (LFR)
The main idea: map each individual, represented as a data point in a given input space, to a probability distribution in a new representation space. The aim of this new representation is to lose any information that can identify whether the person belongs to the protected subgroup, while retaining as much other information as possible.

A discriminative clustering model, where the prototypes act as the clusters

#### Prejudice Remover Regularizer (PR)

ADD DETAILS


### Data Pre-processing

In [2]:
import csv
import pandas as pd

import numpy as np
from scipy.special import softmax
import scipy.optimize as optim
from sklearn.preprocessing import StandardScaler

In [24]:
data = 'https://raw.githubusercontent.com/juliamblake1/ADS-Project-4/main/data/compas-scores-two-years_processed.csv'

In [25]:
compas_scores = pd.read_csv(data,header =0)
compas_scores.head()

,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,...,score_text,v_decile_score,v_score_text,priors_count.1,start,end,event,custody_duration,jail_duration_hours,two_year_recid
0,Male,Greater than 45,Other,0,1,0,0,0,-1.0,1.0,...,Low,1,Low,0,0,327,0,7.0,23.627222,0
1,Male,25 - 45,African-American,0,3,0,0,0,-1.0,1.0,...,Low,1,Low,0,9,159,1,10.0,241.857222,1
2,Male,Less than 25,African-American,0,4,0,1,4,-1.0,1.0,...,Low,3,Low,4,0,63,0,0.0,26.058333,1
3,Male,25 - 45,Other,0,1,0,0,0,0.0,0.0,...,Low,1,Low,0,1,853,0,1.0,31.643889,0
4,Male,25 - 45,Caucasian,0,6,0,0,14,-1.0,1.0,...,Medium,2,Low,14,5,40,1,18.0,151.168333,1


In [26]:
compas_scores_race = compas_scores.loc[compas_scores['race'].isin(['Caucasian' ,'African-American'])]
compas_scores_race

,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,...,score_text,v_decile_score,v_score_text,priors_count.1,start,end,event,custody_duration,jail_duration_hours,two_year_recid
1,Male,25 - 45,African-American,0,3,0,0,0,-1.0,1.0,...,Low,1,Low,0,9,159,1,10.0,241.857222,1
2,Male,Less than 25,African-American,0,4,0,1,4,-1.0,1.0,...,Low,3,Low,4,0,63,0,0.0,26.058333,1
4,Male,25 - 45,Caucasian,0,6,0,0,14,-1.0,1.0,...,Medium,2,Low,14,5,40,1,18.0,151.168333,1
6,Female,25 - 45,Caucasian,0,1,0,0,0,-1.0,1.0,...,Low,1,Low,0,2,747,0,3.0,70.886667,0
7,Male,Less than 25,Caucasian,0,3,0,0,1,428.0,308.0,...,Low,5,Medium,1,0,428,1,1.0,23.719444,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893,Male,25 - 45,African-American,0,2,0,0,0,-1.0,1.0,...,Low,2,Low,0,0,529,1,0.0,22.444167,1
6894,Male,Less than 25,African-American,0,9,0,0,0,-1.0,1.0,...,High,9,High,0,0,169,0,20.0,20.930833,0
6895,Male,Less than 25,African-American,0,7,0,0,0,-1.0,1.0,...,Medium,5,Medium,0,1,860,0,2.0,45.681389,0
6896,Male,Less than 25,African-American,0,3,0,0,0,-1.0,1.0,...,Low,5,Medium,0,1,790,0,2.0,44.832778,0


In [27]:
# converting to binary data
df_one = pd.get_dummies(compas_scores_race["race"])
# print(df_one)

# display result
df_two = pd.concat((df_one, compas_scores_race), axis=1)
df_two = df_two.drop(["race"], axis=1)
df_two = df_two.drop(["Caucasian"], axis=1)
new_df = df_two.rename(columns={"African-American": "race"})
new_df

,race,sex,age_cat,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,...,score_text,v_decile_score,v_score_text,priors_count.1,start,end,event,custody_duration,jail_duration_hours,two_year_recid
1,1,Male,25 - 45,0,3,0,0,0,-1.0,1.0,...,Low,1,Low,0,9,159,1,10.0,241.857222,1
2,1,Male,Less than 25,0,4,0,1,4,-1.0,1.0,...,Low,3,Low,4,0,63,0,0.0,26.058333,1
4,0,Male,25 - 45,0,6,0,0,14,-1.0,1.0,...,Medium,2,Low,14,5,40,1,18.0,151.168333,1
6,0,Female,25 - 45,0,1,0,0,0,-1.0,1.0,...,Low,1,Low,0,2,747,0,3.0,70.886667,0
7,0,Male,Less than 25,0,3,0,0,1,428.0,308.0,...,Low,5,Medium,1,0,428,1,1.0,23.719444,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893,1,Male,25 - 45,0,2,0,0,0,-1.0,1.0,...,Low,2,Low,0,0,529,1,0.0,22.444167,1
6894,1,Male,Less than 25,0,9,0,0,0,-1.0,1.0,...,High,9,High,0,0,169,0,20.0,20.930833,0
6895,1,Male,Less than 25,0,7,0,0,0,-1.0,1.0,...,Medium,5,Medium,0,1,860,0,2.0,45.681389,0
6896,1,Male,Less than 25,0,3,0,0,0,-1.0,1.0,...,Low,5,Medium,0,1,790,0,2.0,44.832778,0


In [28]:
new_df.two_year_recid.value_counts(), new_df.race.value_counts()

(0    3083
 1    2825
 Name: two_year_recid, dtype: int64,
 1    3534
 0    2374
 Name: race, dtype: int64)

In [29]:
 new_df.v_decile_score.value_counts(), new_df.priors_count.value_counts(),

(1     1436
 2      862
 3      819
 4      673
 5      582
 6      511
 7      397
 8      265
 9      253
 10     110
 Name: v_decile_score, dtype: int64,
 0     1679
 1     1090
 2      671
 3      482
 4      340
 5      285
 6      208
 7      186
 8      162
 9      131
 10     102
 11      90
 13      72
 12      66
 14      50
 15      48
 16      36
 17      32
 19      27
 18      24
 21      22
 22      20
 20      17
 23      15
 24      10
 25       8
 28       7
 27       7
 26       5
 29       5
 33       3
 30       2
 38       2
 36       1
 37       1
 35       1
 31       1
 Name: priors_count, dtype: int64)

## LFR Implementation
First the loss functions

In [30]:
np.random.seed(509)


def loss_x(x_new, x_initial):
    """
    Constrains the mapping to Z to be good description of X.
    Prototpyes should retain as much initial info as possible.

    difference is measured by squared sum of difference


    ARGS:
    x_new - Prototypes
    x_initial - raw data
    """
    return np.mean(np.sum(np.square((x_new - x_initial))))


def loss_y(y_true, y_predicted):
    """
    This loss term requires that the prediction of y is as accurate as possible:

    Computes log loss

    ARGS:
    y_true - (num_examples, )
    y_predicted - (num_examples, )
    """
    # logarithm is undefined in 0 which means y cant be 0 or 1 => we clip it
    y_true = np.clip(y_true, 1e-6, 0.999)
    y_predicted = np.clip(y_predicted, 1e-6, 0.999)

    log_loss = np.sum(y_true * np.log(y_predicted) +
                      (1. - y_true) * np.log(1. - y_predicted)) / len(y_true)

    return -log_loss


def loss_z(M_k_sensitive, M_k_non_sensitive):
    """
    Ensures statistical parity

    Calculates L1 distance

    Args:
    M_k_sensitive - (num_prototypes, )
    M_k_non_sensitive - (num_prototypes, )
    """
    return np.sum(np.abs(M_k_sensitive - M_k_non_sensitive))

In [31]:
def distances(X, v, alpha):
    """
    Calculates distance between initial data and each of the prototypes
    Formula -> euclidean(x, v * alpha) (alpha is weight for each feature)

    ARGS:
    X - (num_examples, num_features)
    v - (num_prototypes, num_features)
    alpha - (num_features, 1)

    returns:
    dists - (num_examples, num_prototypes)
    """
    num_examples = X.shape[0]
    num_prototypes = v.shape[0]
    dists = np.zeros(shape=(num_examples, num_prototypes))

    # X = X.values  # converting to NumPy, this is needed in case you pass dataframe
    for i in range(num_examples):
        dist = np.square(X[i] - v)  # squarred distance
        dist_alpha = np.multiply(dist, alpha)  # multiplying by weights
        sum_ = np.sum(dist_alpha, axis=1)
        dists[i] = sum_

    return dists

In [32]:
def M_nk(dists):
    """
    define Mn,k as the probability that x maps to v

    Given the definitions of the prototypes as points in
    the input space, a set of prototypes induces a natural
    probabilistic mapping from X to Z via the softmax

    Since we already have distances calcutated we just map them to probabilities

    NOTE:
    minus distance because smaller the distance better the mapping

    ARGS:
    dists - (num_examples, num_prototypes)

    Return :
    mappings - (num_examples, num_prototypes)
    """
    return softmax(-dists, axis=1)  # specifying axis is important


def M_k(M_nk):
    """
    Calculate mean of the mapping for each prototype

    ARGS:
    M_nk - (num_examples, num_prototypes)

    Returns:
    M_k - mean of the mappings (num_prototypes, )
    """
    return np.mean(M_nk, axis=0)

In [33]:
def x_n_hat(M_nk, v):
    """
    Gets new representation of the data,
    Performs simple dot product

    ARGS:
    M_nk - (num_examples, num_prototypes)
    v - (num_prototypes, num_features)

    Returns:
    x_n_hat - (num_examples, num_features)
    """
    return M_nk @ v


def y_hat(M_nk, w):
    """
    Function calculates labels in the new representation space
    Performs simple dot product

    ARGS:
    M_nk - (num_examples, num_prototypes)
    w - (num_prototypes, )

    returns:
    y_hat - (num_examples, )
    """
    return M_nk @ w

Now for the optimization

In [34]:
def optim_objective(params, data_sensitive, data_non_sensitive, y_sensitive,
                    y_non_sensitive,  inference=False, NUM_PROTOTYPES=10, A_x=0.01, A_y=0.1, A_z=0.5,
                    print_every=100):
    """
    Function gathers all the helper functions to calculate overall loss

    This is further passed to l-bfgs optimizer

    ARGS:
    params - vector of length (2 * num_features + NUM_PROTOTYPES + NUM_PROTOTYPES * num_features)
    data_sensitive - instances belonging to senstive group (num_sensitive_examples, num_features)
    data_non_sensitive - similar to data_sensitive (num_non_senitive_examplesm num_features)
    y_sensitive - labels for sensitive group (num_sensitive_examples, )
    y_non_sensitive - similar to y_sensitive
    inference - (optional) if True than will return new dataset instead of loss
    NUM_PROTOTYPES - (optional), default 10
    A_x - (optional) hyperparameters for loss_X, default 0.01
    A_y - (optional) hyperparameters for loss_Y, default 1
    A_z - (optional) hyperparameters for loss_Z, default 0.5
    print_every - (optional) how often to print loss, default 100
    returns:
    if inference - False :
    float - A_x * L_x + A_y * L_y + A_z * L_z
    if inference - True:
    x_hat_sensitive, x_hat_non_sensitive, y_hat_sensitive, y_hat_non_sensitive
    """
    optim_objective.iters += 1

    num_features = data_sensitive.shape[1]
    # extract values for each variable from params vector
    alpha_non_sensitive = params[:num_features]
    alpha_sensitive = params[num_features:2 * num_features]
    w = params[2 * num_features:2 * num_features + NUM_PROTOTYPES]
    v = params[2 * num_features + NUM_PROTOTYPES:].reshape(NUM_PROTOTYPES, num_features)

    dists_sensitive = distances(data_sensitive, v, alpha_sensitive)
    dists_non_sensitive = distances(data_non_sensitive, v, alpha_non_sensitive)

    # get probabilities of mappings
    M_nk_sensitive = M_nk(dists_sensitive)
    M_nk_non_sensitive = M_nk(dists_non_sensitive)

    # M_k only used for calcilating loss_y(statistical parity)
    M_k_sensitive = M_k(M_nk_sensitive)
    M_k_non_sensitive = M_k(M_nk_non_sensitive)
    L_z = loss_z(M_k_sensitive, M_k_non_sensitive)  # stat parity

    # get new representation of data
    x_hat_sensitive = x_n_hat(M_nk_sensitive, v)
    x_hat_non_sensitive = x_n_hat(M_nk_non_sensitive, v)
    # calculates how close new representation is to original data
    L_x_sensitive = loss_x(data_sensitive, x_hat_sensitive)
    L_x_non_sensitive = loss_x(data_non_sensitive, x_hat_non_sensitive)

    # get new values for labels
    y_hat_sensitive = y_hat(M_nk_sensitive, w)
    y_hat_non_sensitive = y_hat(M_nk_non_sensitive, w)
    # ensure how good new predictions are(log_loss)
    L_y_sensitive = loss_y(y_sensitive, y_hat_sensitive)
    L_y_non_sensitive = loss_y(y_non_sensitive, y_hat_non_sensitive)

    L_x = L_x_sensitive + L_x_non_sensitive
    L_y = L_y_sensitive + L_y_non_sensitive

    loss = A_x * L_x + A_y * L_y + A_z * L_z

    if optim_objective.iters % print_every == 0:
        print(f'loss on iteration {optim_objective.iters} : {loss}, L_x - {L_x * A_x} L_y - {L_y * A_y} L_z - {L_z * A_z}')

    if not inference:
        return loss
    if inference:
        return x_hat_sensitive, x_hat_non_sensitive, y_hat_sensitive, y_hat_non_sensitive

optim_objective.iters = 0

Now lets actually implement after we have all of the functions necessary

We are going to use a subset of the data for trial:

In [35]:
use_df = new_df[['two_year_recid', 'race', 'priors_count', 'v_decile_score', 'age_cat']]
use_df

,two_year_recid,race,priors_count,v_decile_score,age_cat
1,1,1,0,1,25 - 45
2,1,1,4,3,Less than 25
4,1,0,14,2,25 - 45
6,0,0,0,1,25 - 45
7,1,0,1,5,Less than 25
...,...,...,...,...,...
6893,1,1,0,2,25 - 45
6894,0,1,0,9,Less than 25
6895,0,1,0,5,Less than 25
6896,0,1,0,5,Less than 25


In [36]:
RACE_SENSITIVE = 0

# seperation into sensitive and non sensitive
data_sensitive = use_df[use_df.race > RACE_SENSITIVE]
data_non_sensitive = use_df[use_df.race <= RACE_SENSITIVE]
y_sensitive = data_sensitive.two_year_recid
y_non_sensitive = data_non_sensitive.two_year_recid

In [37]:
print (f'Dataset contains {use_df.shape[0]} examples and {use_df.shape[1]} features')
print (f'From which {data_sensitive.shape[0]} belong to sensitive group and {data_non_sensitive.shape[0]} to non sensitive group ')

Dataset contains 5908 examples and 5 features
From which 3534 belong to sensitive group and 2374 to non sensitive group 


In [38]:
from sklearn.preprocessing import StandardScaler

# Remove the target variable
del data_sensitive['two_year_recid']
del data_non_sensitive['two_year_recid']

# Convert non-numeric columns to numeric format or remove these columns
data_sensitive = data_sensitive.select_dtypes(include=[np.number])
data_non_sensitive = data_non_sensitive.select_dtypes(include=[np.number])

# Standard Scaling
scaler_sensitive = StandardScaler()
data_sensitive_scaled = scaler_sensitive.fit_transform(data_sensitive)

scaler_non_sensitive = StandardScaler()
data_non_sensitive_scaled = scaler_non_sensitive.fit_transform(data_non_sensitive)


In [39]:
#del data_sensitive['two_year_recid']
#del data_non_sensitive['two_year_recid']

# Standard Scaling
data_sensitive = StandardScaler().fit_transform(data_sensitive)
data_non_sensitive = StandardScaler().fit_transform(data_non_sensitive)


In [40]:
NUM_PROTOTYPES = 10
num_features = data_sensitive.shape[1]

params = np.random.uniform(size=(num_features * 2 + NUM_PROTOTYPES + NUM_PROTOTYPES * num_features))
# here we generate random weight for each of the features both for sensitive data
# and for non sensitive, hence num_features*2(in paper this is denoted as alpha)
# alphas are used for calculating distances


In [41]:
# Then NUM_PROTOTYPES is a weight for each prototype, this is multiplied with
# M_nk s and used for calculating y_hat

# Next is NUM_PROTOTYPES * num_features which is v(in paper), this is also used
# for calculating distances


bnd = [] # This is needed for l-bfgs algorithm
for i, _ in enumerate(params):
    if i < num_features * 2 or i >= num_features * 2 + NUM_PROTOTYPES:
        bnd.append((None, None))
    else:
        bnd.append((0, 1))

In [44]:
# First, find the optimized parameters
new_params = optim.fmin_l_bfgs_b(optim_objective, x0=params, epsilon=1e-5,
                                 args=(data_sensitive, data_non_sensitive,
                                       y_sensitive, y_non_sensitive),
                                 bounds=bnd, approx_grad=True, maxfun=1000,
                                 maxiter=1000)[0]

# Then, call optim_objective with 'inference=True' to get the transformed datasets
x_hat_senitive, x_hat_nons, y_hat_sens, y_hat_nons = optim_objective(new_params, data_sensitive, data_non_sensitive,
                                                                     y_sensitive, y_non_sensitive, inference=True)


loss on iteration 1100 : 80.73268935260076, L_x - 80.52780169226443 L_y - 0.1667604493635705 L_z - 0.03812721097275415
loss on iteration 1200 : 64.84061883656207, L_x - 64.49645461806863 L_y - 0.19261180858962068 L_z - 0.15155240990382776
loss on iteration 1300 : 27.850807181200793, L_x - 27.49950041248014 L_y - 0.20105602735878794 L_z - 0.1502507413618665
loss on iteration 1400 : 17.49702791668779, L_x - 17.2529526010648 L_y - 0.17287679504911047 L_z - 0.07119852057388514
loss on iteration 1500 : 12.760923227485874, L_x - 12.520160338055907 L_y - 0.1777995359712376 L_z - 0.06296335345872899
loss on iteration 1600 : 10.180133840496916, L_x - 9.974065658230854 L_y - 0.1600277603374338 L_z - 0.0460404219286282
loss on iteration 1700 : 7.685111294498495, L_x - 7.462540857893822 L_y - 0.15894319617237526 L_z - 0.06362724043229781
loss on iteration 1800 : 5.911134451733174, L_x - 5.700565983354602 L_y - 0.14909541455756 L_z - 0.06147305382101177
loss on iteration 1900 : 436.12194930636315, 

In [45]:
print ('Done')

Done
